# compa

> Retrieve and process data from WRDS Compustat Annual

In [ ]:
#| default_exp wrds.compa

In [ ]:
#|exports
from __future__ import annotations
from typing import List

import pandas as pd
import numpy as np

import pandasmore as pdm
from finsets.wrds import wrds_api

In [ ]:
#| exports
PROVIDER = 'Wharton Research Data Services (WRDS)'
URL = 'https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/crspcompustat-merged/fundamentals-annual/'
LIBRARY = 'comp'
TABLE = 'funda'
LINK_LIBRARY = 'crsp'
LINK_TABLE = 'ccmxpf_lnkhist'
FREQ = 'A'
MIN_YEAR = 1950
MAX_YEAR = None
ENTITY_ID_IN_RAW_DSET = 'permno'
ENTITY_ID_IN_CLEAN_DSET = 'permno'
TIME_VAR_IN_RAW_DSET = 'datadate'
TIME_VAR_IN_CLEAN_DSET = 'Adate'

In [ ]:
#| export
def list_all_vars() -> pd.DataFrame:
    "Collects names of all available variables from WRDS f`{LIBRARY}.{TABLE}`"

    try:
        db = wrds_api.Connection()
        funda = db.describe_table(LIBRARY,TABLE).assign(wrds_library=LIBRARY, wrds_table=TABLE)
    finally:
        db.close()

    return funda[['name','type','wrds_library','wrds_table']]

In [ ]:
#| eval: false
all_vars = list_all_vars()

In [ ]:
#| eval: false
all_vars.name.count()

948

In [ ]:
#| export
def default_raw_vars():
    """Defines default variables used in `get_raw_data` if none are specified."""

    return ['datadate', 'gvkey', 'cusip' ,'cik' ,'tic' ,'fyear' ,'fyr' ,'naicsh', 'sich' ,'exchg',  
            'lt' ,'at' ,'txditc' ,'pstkl' ,'pstkrv' ,'pstk' ,'csho' ,'ajex' , 'rdip',
            'act' ,'dvc' ,'xad','seq' ,'che' ,'lct' ,'dlc' ,'ib' ,'dvp' ,'txdi' ,'dp' ,
            'txp' ,'oancf' ,'ivncf' ,'fincf' ,'dltt' ,'mib','ceq' ,'invt' ,'cogs' , 'revt',
            'sale' ,'capx' ,'xrd' ,'txdb' ,'prcc_f' ,'sstk' ,'prstkc' ,'dltis' ,'dltr' ,'emp' ,
            'dd1' ,'ppegt' ,'ppent' ,'xint' ,'txt' ,'sppe' ,'gdwl' ,'xrent' ,'re' ,'dvpsx_f' ,
            'tstk' ,'wcap' ,'rect' ,'xsga' ,'aqc' ,'oibdp' ,'dpact' ,'fic' ,'ni' ,'ivao' ,'ivst' ,
            'dv' , 'intan' ,'pi' ,'txfo' ,'pifo' ,'xpp' ,'drc' ,'drlt' ,'ap' ,'xacc' ,'itcb']             

In [ ]:
print(default_raw_vars())

['datadate', 'gvkey', 'cusip', 'cik', 'tic', 'fyear', 'fyr', 'naicsh', 'sich', 'exchg', 'lt', 'at', 'txditc', 'pstkl', 'pstkrv', 'pstk', 'csho', 'ajex', 'rdip', 'act', 'dvc', 'xad', 'seq', 'che', 'lct', 'dlc', 'ib', 'dvp', 'txdi', 'dp', 'txp', 'oancf', 'ivncf', 'fincf', 'dltt', 'mib', 'ceq', 'invt', 'cogs', 'revt', 'sale', 'capx', 'xrd', 'txdb', 'prcc_f', 'sstk', 'prstkc', 'dltis', 'dltr', 'emp', 'dd1', 'ppegt', 'ppent', 'xint', 'txt', 'sppe', 'gdwl', 'xrent', 're', 'dvpsx_f', 'tstk', 'wcap', 'rect', 'xsga', 'aqc', 'oibdp', 'dpact', 'fic', 'ni', 'ivao', 'ivst', 'dv', 'intan', 'pi', 'txfo', 'pifo', 'xpp', 'drc', 'drlt', 'ap', 'xacc', 'itcb']


In [ ]:
#| export
def parse_varlist(vars: List[str]|str=None, #list of variables requested by user
                  req_vars: List[str] = ['gvkey', 'datadate'], #list of variables that will get downloaded, even if not in `vars`
                  prefix: str='a.', #string to add in front of each variable name when we build the SQL string of variable names
                  ) -> str:
    """Adds required variables to requested variables, validates them, and builds the SQL string with their names"""

    if vars=='*': return f'{prefix}*' 

    # Build full list of variables that will be downloaded
    if vars is None: vars = default_raw_vars()
    if req_vars is None: req_vars = []
    vars =  req_vars + [x for x in vars if x not in req_vars] #in case `vars` already contains some of the required variables

    # Validate variables to be downloaded (make sure that they are in the target database)
    valid_vars = list(list_all_vars().name)
    invalid_vars = [v for v in vars if v not in valid_vars]
    if invalid_vars: raise ValueError(f"These vars are not in the database: {invalid_vars}") 

    return ','.join([f'{prefix}{var_name}' for var_name in vars])

In [ ]:
#| eval: false
parse_varlist(['at','lt'])

'a.gvkey,a.datadate,a.at,a.lt'

In [ ]:
#| export
def get_raw_data(
        vars: List[str]=None, # If None, downloads `default_raw_vars`; use '*' to get all available variables
        nrows: int=None, #Number of rows to download. If None, full dataset will be downloaded
        start_date: str=None, # Start date in MM/DD/YYYY format
        end_date: str=None #End date in MM/DD/YYYY format
) -> pd.DataFrame:
    """Downloads `vars` from `start_date` to `end_date` from WRDS `{LIBRARY}.{TABLE}` library and adds PERMNO and PERMCO as in CCM"""
 
    vars = parse_varlist(vars, prefix='a.')

    sql_string=f"""SELECT b.lpermno as permno, b.lpermco as permco, b.liid as iid, b.linkprim as linkprim, {vars}
                    FROM {LIBRARY}.{TABLE} AS a
                    INNER JOIN {LINK_LIBRARY}.{LINK_TABLE} AS b ON a.gvkey = b.gvkey
                    WHERE datadate BETWEEN b.linkdt AND COALESCE(b.linkenddt, CURRENT_DATE)
                            AND b.linktype IN ('LU','LC') 
                            AND indfmt='INDL' AND datafmt='STD' AND popsrc='D' AND consol='C'
                """
    if start_date is not None: sql_string += r" AND datadate >= %(start_date)s"
    if end_date is not None: sql_string += r" AND datadate <= %(end_date)s"
    if nrows is not None: sql_string += r" LIMIT %(nrows)s"
    
    return wrds_api.download(sql_string,
                             params={'start_date':start_date, 'end_date':end_date, 'nrows':nrows})

The `get_raw_data` function will produce identical results to the ones we would obtain if we used the WRDS website (with the default options unchanged). 

It results in no `permno-datadate` duplicates, but there is a small number of `gvkey-datadate` duplicates (about 1% of the data) because each `permno` maps to a unique `gvkey+iid` value and some gvkeys have multiple share classes (different iid's). 

Finally, not that you might want to restrict yourself to `linkprim='P'` (i.e. links only to the primary equity security issued by the firm) which retains about 80% of the data, or `linkprim in ('P','C')` which retains 99% of the data. The latter also results in unique `gvkey-datadate` records which is why we use it as the default option in the `process_raw_data` function.

In [ ]:
#| eval: false
r = get_raw_data(vars='*', nrows=1)
r

,permno,permco,iid,linkprim,gvkey,datadate,fyear,indfmt,consol,popsrc,...,prcc_f,prch_f,prcl_f,adjex_f,rank,au,auop,auopic,ceoso,cfoso
0,25881.0,23369.0,01,P,001000,1970-12-31,1970.0,INDL,C,D,...,10.0,10.875,7.5,1.0,None,None,None,None,None,None


In [ ]:
#| eval: false
raw = get_raw_data(vars = ['at', 'lt'], start_date='01/01/2021', end_date='01/01/2022', nrows=1)

In [ ]:
#| eval: false
raw.head(0)

,permno,permco,iid,linkprim,gvkey,datadate,at,lt


In [ ]:
#| eval: false
raw = get_raw_data(start_date='01/01/2021', end_date='01/01/2023')

In [ ]:
#| eval: false
raw.head(0)

,permno,permco,iid,linkprim,gvkey,datadate,cusip,cik,tic,fyear,...,intan,pi,txfo,pifo,xpp,drc,drlt,ap,xacc,itcb


In [ ]:
#| export
def process_raw_data(
        df: pd.DataFrame=None,  # Must contain `permno` and `datadate` columns   
        linkprim_filter: list=['P','C'],      
        clean_kwargs: dict={},  # Params to pass to `pdm.setup_panel` other than `panel_ids`, `time_var`, and `freq`
) -> pd.DataFrame:
    """Applies `pandasmore.setup_panel` to `df`"""

    if linkprim_filter: df = df.loc[df['linkprim'].isin(linkprim_filter)].copy()

    df = pdm.setup_panel(df, panel_ids=ENTITY_ID_IN_RAW_DSET, time_var=TIME_VAR_IN_RAW_DSET, freq=FREQ, **clean_kwargs)
    return df 

In [ ]:
#| eval: false
df_clean = process_raw_data(raw)

In [ ]:
#| eval: false
df_clean.head(0)

,,datadate,dtdate,permco,iid,linkprim,gvkey,cusip,cik,tic,fyear,...,intan,pi,txfo,pifo,xpp,drc,drlt,ap,xacc,itcb
permno,Adate,,,,,,,,,,,,,,,,,,,,,


In [ ]:
#| export
def features(df: pd.DataFrame=None
             ) -> pd.DataFrame:

    out = pd.DataFrame(index=df.index)

    out['lag_at'] = pdm.lag(df['at'])

    # book equity vars
    out['pstk0'] = df['pstk'].fillna(0)
    out['pref_stock'] = np.where(df['pstkrv'].isnull(), df['pstkl'], df['pstkrv'])
    out['pref_stock'] = np.where(out['pref_stock'].isnull(),out['pstk0'], out['pref_stock'])
    out['shreq'] = np.where(df['seq'].isnull(), df['ceq'] + out['pstk0'], df['seq'])
    out['shreq'] = np.where(out['shreq'].isnull(), df['at'] - df['lt'], out['shreq'])
    out['bookeq'] = out['shreq'] + df['txditc'].fillna(0) - out['pref_stock']
    out['bookeq_w_itcb'] = out['bookeq'] + df['itcb'].fillna(0)

    out['tobinq'] = (df['at'] - out['bookeq'] + df['prcc_f'] * df['csho']) / df['at']

    # issuance vars
    out['equityiss_tot'] = (pdm.rdiff(out['bookeq']) - pdm.rdiff(df['re'])) 
    out['equityiss_cfs'] = (df['sstk'].fillna(0) - df['prstkc'].fillna(0))
    out['debtiss_tot'] = (pdm.rdiff(df['at']) - pdm.rdiff(out['bookeq'])) 
    out['debtiss_cfs'] = (df['dltis'].fillna(0) - df['dltr'].fillna(0)) 
    out['debtiss_bs'] = (pdm.rdiff(df['dltt']) + pdm.rdiff(df['dlc'].fillna(0))) 
    for v in ['equityiss_tot','equityiss_cfs','debtiss_tot','debtiss_cfs','debtiss_bs']:
        out[f'{v}_2la'] = out[v] / out['lag_at']

    # investment vars
    out['ppent_pch'] = pdm.rpct_change(df['ppent'])
    out['capx_2la'] = df['capx'] / out['lag_at']

    # profitability vars
    out['roa'] = df['ib'] / df['at']

    # cash flow vars
    out['cflow_is'] = (df['ib']+df['dp']) 
    out['cflow_cfs'] = df['oancf'] 
    out['cflow_full'] = np.where(df.dtdate.dt.year<1987, out['cflow_is'], out['cflow_cfs'])
    for v in ['cflow_is','cflow_cfs','cflow_full']:
        out[f'{v}_2la'] = out[v] / out['lag_at']

    # liquidity vars
    out['cash_2a'] = df['che'] / df['at']

    # leverage vars
    out['booklev'] = (df['dltt'] + df['dlc']) / df['at']
    out.loc[out.booklev<0, 'booklev'] = 0
    out.loc[out.booklev>1, 'booklev'] = 1

    # payout vars
    out['dividends_2la'] = df['dvc'].fillna(0) / out['lag_at']
    out['repurchases_2la'] = df['prstkc'].fillna(0) / out['lag_at']

    out = out.replace([np.inf, -np.inf], np.nan)
    return out 


In [ ]:
#| eval: false
ftrs = features(df_clean)

In [ ]:
#| eval: false
ftrs.head(0)

,,lag_at,pstk0,pref_stock,shreq,bookeq,bookeq_w_itcb,tobinq,equityiss_tot,equityiss_cfs,debtiss_tot,...,cflow_is,cflow_cfs,cflow_full,cflow_is_2la,cflow_cfs_2la,cflow_full_2la,cash_2a,booklev,dividends_2la,repurchases_2la
permno,Adate,,,,,,,,,,,,,,,,,,,,,


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()